In [2]:
##合并lora和基础权重,合并完成之后不用再次运行这段代码！！！
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "/root/autodl-tmp/DeepSeek-R1-Distill-Qwen-7B"  # 替换为你的模型路径
lora_path = "./DeepSeek-R1-Distill-Qwen-7B_LoRA"  # 替换为你的 LoRA 适配器路径

# 加载基础模型和 LoRA 适配器
base_model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)  # 加载 tokenizer
lora_model = PeftModel.from_pretrained(base_model, lora_path)

# 合并 LoRA 适配器到基础模型
merged_model = lora_model.merge_and_unload()

# 保存合并后的模型和 tokenizer
merged_model.save_pretrained("./merged_model")  # 保存合并后的模型
tokenizer.save_pretrained("./merged_model")  # 保存 tokenizer

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

('./merged_model/tokenizer_config.json',
 './merged_model/special_tokens_map.json',
 './merged_model/tokenizer.json')

In [ ]:
import vllm
from vllm.sampling_params import SamplingParams

# 初始化模型
def load_model(model_name):
    try:
        model = vllm.LLM(model_name)
        print("模型加载成功！")
        return model
    except Exception as e:
        print(f"加载模型失败: {e}")
        return None

# 初始化对话历史
def initialize_history():
    return []

# 格式化对话历史
def format_history(history):
    formatted_history = "\n".join([f"User: {item}" if i % 2 == 0 else f"Assistant: {item}" for i, item in enumerate(history)])
    return formatted_history

# 生成完整的提示，包含历史对话和当前用户输入
def generate_prompt(prompt, history):
    formatted_history = format_history(history)
    full_prompt = f"{formatted_history}\n\nUser: {prompt}\n\nAssistant:"
    return full_prompt

# 生成回答并更新历史记录
def generate_response(model, prompt, history):
    try:
        sampling_params = SamplingParams(
            max_tokens=1280,
            temperature=0.7,
            top_p=0.9,
            stop=["\n\nUser:", "\n\nAssistant:"]  # 使用 `stop` 参数避免生成多余内容
        )
        output = model.generate([prompt], sampling_params=sampling_params)
        raw_response = output[0].outputs[0].text.strip()
        return raw_response
    except Exception as e:
        print(f"生成回答失败: {e}")
        return None


# 主程序
model_name = "./merged_model"  # 替换为合并后的模型路径
model = load_model(model_name)
if model is None:
    print("模型加载失败，检查模型路径是否正确")
else:
    dialogue_history = initialize_history()
    print("模型已加载，现在可以开始提问了！输入 'exit' 退出。")
    while True:
        # 打印对话历史
        # print("=======对话历史=====")
        # print(format_history(dialogue_history))
        # print("=========")
        
        user_input = input("\n请输入问题: ")
        if user_input.lower() == 'exit':
            print("退出程序。")
            break
        
        # 生成完整的提示
        prompt = generate_prompt(user_input, dialogue_history)
        
        # 生成回答
        response = generate_response(model, prompt, dialogue_history)
        if response is not None:
            print(f"\nAssistant: {response}")
            # 更新对话历史
            dialogue_history.append(user_input)
            dialogue_history.append(response)

INFO 02-19 23:55:13 __init__.py:190] Automatically detected platform cuda.
INFO 02-19 23:55:14 config.py:2382] Downcasting torch.float32 to torch.float16.
INFO 02-19 23:55:20 config.py:542] This model supports multiple tasks: {'classify', 'score', 'embed', 'generate', 'reward'}. Defaulting to 'generate'.
WARNING 02-19 23:55:20 arg_utils.py:1135] Chunked prefill is enabled by default for models with max_model_len > 32K. Currently, chunked prefill might not work with some features or models. If you encounter any issues, please disable chunked prefill by setting --enable-chunked-prefill=False.
INFO 02-19 23:55:20 config.py:1556] Chunked prefill is enabled with max_num_batched_tokens=2048.
INFO 02-19 23:55:20 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.2) with config: model='./merged_model', speculative_config=None, tokenizer='./merged_model', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=Fal

Loading safetensors checkpoint shards:   0% Completed | 0/7 [00:00<?, ?it/s]


INFO 02-19 23:55:26 model_runner.py:1115] Loading model weights took 14.2717 GB
INFO 02-19 23:55:27 worker.py:267] Memory profiling takes 0.89 seconds
INFO 02-19 23:55:27 worker.py:267] the current vLLM instance can use total_gpu_memory (47.54GiB) x gpu_memory_utilization (0.90) = 42.78GiB
INFO 02-19 23:55:27 worker.py:267] model weights take 14.27GiB; non_torch_memory takes 0.06GiB; PyTorch activation peak memory takes 1.40GiB; the rest of the memory reserved for KV Cache is 27.05GiB.
INFO 02-19 23:55:27 executor_base.py:110] # CUDA blocks: 31659, # CPU blocks: 4681
INFO 02-19 23:55:27 executor_base.py:115] Maximum concurrency for 131072 tokens per request: 3.86x
INFO 02-19 23:55:29 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory

Capturing CUDA graph shapes: 100%|██████████| 35/35 [00:13<00:00,  2.66it/s]

INFO 02-19 23:55:43 model_runner.py:1562] Graph capturing finished in 13 secs, took 0.80 GiB
INFO 02-19 23:55:43 llm_engine.py:431] init engine (profile, create kv cache, warmup model) took 16.51 seconds


...

Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  2.01it/s, est. speed input: 86.58 toks/s, output: 38.25 toks/s]



Assistant: 好，我这就去，还不快去快去，别耽误了。



请输入问题:  谢了


Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  4.46it/s, est. speed input: 312.84 toks/s, output: 35.75 toks/s]



Assistant: 不谢，快去吧。
